## AIDevs2 tasks from newest at the top to oldest at the bottom ,updated daily, to be cleaned ;)

In [152]:
import openai
import requests
from utilities.common import OpenAIClient, AIDevsClient
from utilities.config import AI_DEVS_SERVER, AI_DEVS_API_KEY, OPEN_AI_API_KEY

BASE_URL = AI_DEVS_SERVER
API_KEY = AI_DEVS_API_KEY
openai.api_key  = OPEN_AI_API_KEY

AIDevs_Client = AIDevsClient(base_url=BASE_URL, api_key=API_KEY)

#!pip install --upgrade openai
!pip install python-dotenv



# inprompt

Skorzystaj z API zadania.aidevs.pl, aby pobrać dane zadania inprompt. Znajdziesz w niej dwie właściwości — input, czyli tablicę / listę zdań na temat różnych osób (każde z nich zawiera imię jakiejś osoby) oraz question będące pytaniem na temat jednej z tych osób. Lista jest zbyt duża, aby móc ją wykorzystać w jednym zapytaniu, więc dowolną techniką odfiltruj te zdania, które zawierają wzmiankę na temat osoby wspomnianej w pytaniu. Ostatnim krokiem jest wykorzystanie odfiltrowanych danych jako kontekst na podstawie którego model ma udzielić odpowiedzi na pytanie. Zatem: pobierz listę zdań oraz pytanie, skorzystaj z LLM, aby odnaleźć w pytaniu imię, programistycznie lub z pomocą no-code odfiltruj zdania zawierające to imię. Ostatecznie spraw by model odpowiedział na pytanie, a jego odpowiedź prześlij do naszego API w obiekcie JSON zawierającym jedną właściwość “answer”.

In [153]:
task_name = "inprompt"

In [154]:
token = AIDevs_Client.get_token(task_name)['token']

task = AIDevs_Client.get_task(token)
people = task['input']
question = task['question']

question

'co lubi jeść na śniadanie Alojzy?'

In [155]:
#Extract name from the question

prompt = f"""
Extract name from the question, nothing more:
{question}
"""

name = OpenAIClient.get_completion(prompt, model="gpt-3.5-turbo")

name

'Alojzy'

In [156]:
#Find list item with contex about name

contex = [sentence for sentence in people if name in sentence]

contex

['Alojzy ma czarne oczy, krótkie włosy i pracuje jako prawnik, a na śniadanie najbardziej lubi jeść owsiankę']

In [157]:
#Answer the question

prompt = f"""
{contex}, {question}
"""

answer = OpenAIClient.get_completion(prompt)
answer

'Alojzy najbardziej lubi jeść owsiankę na śniadanie.'

In [158]:
result = AIDevs_Client.submit_answer(token, answer)
result

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# liar

Jest to mechanizm, który mówi nie na temat w 1/3 przypadków. Twoje zadanie polega na tym, aby do endpointa /task/ wysłać swoje pytanie w języku angielskim (dowolne, np “What is capital of Poland?’) w polu o nazwie ‘question’ (metoda POST, jako zwykłe pole formularza, NIE JSON). System API odpowie na to pytanie (w polu ‘answer’) lub zacznie opowiadać o czymś zupełnie innym, zmieniając temat. Twoim zadaniem jest napisanie systemu filtrującego (Guardrails), który określi (YES/NO), czy odpowiedź jest na temat. Następnie swój werdykt zwróć do systemu sprawdzającego jako pojedyncze słowo YES/NO. Jeśli pobierzesz treść zadania przez API bez wysyłania żadnych dodatkowych parametrów, otrzymasz komplet podpowiedzi. Skąd wiedzieć, czy odpowiedź jest ‘na temat’? Jeśli Twoje pytanie dotyczyło stolicy Polski, a w odpowiedzi otrzymasz spis zabytków w Rzymie, to odpowiedź, którą należy wysłać do API to NO.

In [159]:
task_name = "liar"

In [178]:
question = "What is the moon?"
#question = OpenAIClient.get_completion("respond with random question", model="gpt-3.5-turbo", max_tokens=100, temperature=0.8)

question

'What is the moon?'

In [161]:
AIDevs_Client = AIDevsClient(base_url=BASE_URL, api_key=API_KEY)

In [162]:
def ask_question(token, question):
    url = f"{BASE_URL}/task/{token}"
    payload = {
        "question": question
    }
    response = requests.post(url, data=payload)  # Using data instead of json
    if response.status_code == 200:
        return response.json()  # Return the entire JSON response
    else:
        print(f"Error asking question: {response.text}")
        return None

In [163]:
token = AIDevs_Client.get_token(task_name)['token']

answer_to_question = ask_question(token, question)['answer']

print(question + " : " + answer_to_question)

What is the moon? : The moon is a natural satellite that orbits around the Earth. It is the Earth's only natural satellite and is located about 384,400 kilometers away from our planet. The moon is approximately 4.5 billion years old and has a diameter of about 3,474 kilometers. It plays a significant role in various aspects, including tides, lunar cycles, and even cultural and religious beliefs. The moon also has a barren and rocky surface, with no atmosphere or liquid water.


Guardrails

In [164]:
prompt = f"""
Given the question '{question}' and the answer '{answer_to_question}', is the answer relevant? Respond with YES or NO.
"""

response = OpenAIClient.get_completion(prompt, model="gpt-3.5-turbo", max_tokens=100, temperature=0.2)

response.strip()  # Stripping to ensure there's no extra whitespace

'Yes'

In [165]:
AIDevs_Client.submit_answer(token, response)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}

# blogger

Napisz wpis na bloga (w języku polskim) na temat przyrządzania pizzy Margherity. Zadanie w API nazywa się ”blogger”. Jako wejście otrzymasz spis 4 rozdziałów, które muszą pojawić się we wpisie. Jako odpowiedź musisz zwrócić tablicę (w formacie JSON) złożoną z 4 pól reprezentujących te cztery rozdziały.

In [179]:
task_name = "blogger"

In [180]:
token = AIDevs_Client.get_token(task_name)['token']
task = AIDevs_Client.get_task(token)['blog']

print(task)

['Wstęp: kilka słów na temat historii pizzy', 'Niezbędne składniki na pizzę', 'Robienie pizzy', 'Pieczenie pizzy w piekarniku']


In [181]:
prompt = """
        You are a pizza expert that writes a blog about pizza margherita in polish.
        Write a short paragraph about the given topic:
        """
        
response = OpenAIClient.get_completion(prompt, model="gpt-3.5-turbo", max_tokens=1000, temperature=0.3)


blog_post = [OpenAIClient.get_completion(prompt+topic) for topic in task]

print(blog_post)

['Początki pizzy sięgają starożytnych czasów, kiedy to mieszkańcy Bliskiego Wschodu piekli na płaskich kamieniach chleb, na którym umieszczali różne dodatki. Jednak pizza, którą znamy i kochamy dzisiaj, pochodzi z Neapolu w Italii. W połowie XIX wieku, kiedy to Włochy były zjednoczone, królowa Margherita odwiedziła Neapol. Na jej cześć miejscowy piekarz, Raffaele Esposito, stworzył specjalną pizzę, na której umieścił składniki w barwach włoskiej flagi: czerwone pomidory, białą mozzarellę i zielone liście bazylii. Ta pizza, nazwana na cześć królowej Margherity, stała się prototypem dla wszystkich pizz margherita, które znamy i uwielbiamy dzisiaj.', 'Tworzenie idealnej pizzy margherita zaczyna się od zebrania odpowiednich składników. Na początek potrzebujesz wysokiej jakości mąki typu 00, drożdży, soli i wody do przygotowania podstawy, czyli ciasta na pizzę. Następnie, kluczowym składnikiem jest sos pomidorowy, najlepiej przygotowany z dojrzałych, świeżych pomidorów San Marzano. Kolejnym

In [182]:
result = AIDevs_Client.submit_answer(token, blog_post)

print(result)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


# moderation

Zastosuj wiedzę na temat działania modułu do moderacji treści i rozwiąż zadanie o nazwie “moderation” z użyciem naszego API do sprawdzania rozwiązań. Zadanie polega na odebraniu tablicy zdań (4 sztuki), a następnie zwróceniu tablicy z informacją, które zdania nie przeszły moderacji. Jeśli moderacji nie przeszło pierwsze i ostatnie zdanie, to odpowiedź powinna brzmieć [1,0,0,1]. Pamiętaj, aby w polu ‘answer’ zwrócić tablicę w JSON, a nie czystego stringa.

In [183]:
task_name = "moderation"

In [184]:
token = AIDevs_Client.get_token(task_name)['token']
task = AIDevs_Client.get_task(token)['input']

print(task)

['ten gość musi zginąć. Nie pozwole sobię na obrażanie mnie.', 'Sasha.Grey.s3x.p0rn.extreme-interracial.S03E12.DVDRip.mp4', 'majonez Winiary jest lepszy od Kieleckiego', 'azjaci są głupi i brzydcy i nie powinni żyć']


Moderation

In [172]:
# Initialize an empty list to hold the moderation results
moderation_results = []

# Iterate through each item in the zadanie list
for item in task:
    # Send the item to the Moderation endpoint
    response = openai.Moderation.create(input=item)
    
    # Access the 'flagged' value from the response
    flagged = response['results'][0]['flagged']
    
    # Append 1 if flagged, else 0, to the moderation_results list
    moderation_results.append(1 if flagged else 0)

In [173]:
moderation_results


[1, 1, 1, 0]

In [174]:
result = AIDevs_Client.submit_answer(token, moderation_results)

print(result)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}


# helloapi

Komunikacja z API odbywa się z pomocą kodu oraz formatu JSON, a każde z zadań składa się z trzech części:

autoryzacji

pobierania danych wejściowych (string lub tablica obiektów)

odesłania odpowiedzi (właściwość answer)

In [175]:
task_name = "helloapi"

In [176]:
token = AIDevs_Client.get_token(task_name)['token']
cookie = AIDevs_Client.get_task(token)['cookie']
result = AIDevs_Client.submit_answer(token, cookie)

print(result)


{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}
